### Bibliotheken importeren

In [1]:
import numpy as np
import pandas as pd
import requests as req
from IPython.display import display
from fuzzywuzzy import fuzz
import plotly.graph_objects as go
import geopandas as gpd

### Open Charge Map request

In [2]:
# https://openchargemap.org/site/develop/api
api_key = "bef339a4-319a-4a46-bbbe-a5f13db5bd24"
ocm_url = f"https://api.openchargemap.io/v3/poi/?output=json&key=${api_key}&countrycode=NL&maxresults=100000"
req_ocm = req.get(ocm_url)

### Open Charge Map Data Tabel

In [3]:
df_ocm_raw = pd.json_normalize(req_ocm.json())
pd.set_option("max_columns", None) # show all cols
pd.set_option("max_rows", None) # show all cols
pd.set_option('max_colwidth', None) # show full width of showing cols
pd.set_option("expand_frame_repr", False) # print cols side by side as it's supposed to be
display(df_ocm_raw.head())

,UserComments,PercentageSimilarity,MediaItems,IsRecentlyVerified,DateLastVerified,ID,UUID,ParentChargePointID,DataProviderID,DataProvidersReference,OperatorID,OperatorsReference,UsageTypeID,UsageCost,Connections,NumberOfPoints,GeneralComments,DatePlanned,DateLastConfirmed,StatusTypeID,DateLastStatusUpdate,MetadataValues,DataQualityLevel,DateCreated,SubmissionStatusTypeID,DataProvider.WebsiteURL,DataProvider.Comments,DataProvider.DataProviderStatusType.IsProviderEnabled,DataProvider.DataProviderStatusType.ID,DataProvider.DataProviderStatusType.Title,DataProvider.IsRestrictedEdit,DataProvider.IsOpenDataLicensed,DataProvider.IsApprovedImport,DataProvider.License,DataProvider.DateLastImported,DataProvider.ID,DataProvider.Title,OperatorInfo.WebsiteURL,OperatorInfo.Comments,OperatorInfo.PhonePrimaryContact,OperatorInfo.PhoneSecondaryContact,OperatorInfo.IsPrivateIndividual,OperatorInfo.AddressInfo,OperatorInfo.BookingURL,OperatorInfo.ContactEmail,OperatorInfo.FaultReportEmail,OperatorInfo.IsRestrictedEdit,OperatorInfo.ID,OperatorInfo.Title,UsageType.IsPayAtLocation,UsageType.IsMembershipRequired,UsageType.IsAccessKeyRequired,UsageType.ID,UsageType.Title,StatusType.IsOperational,StatusType.IsUserSelectable,StatusType.ID,StatusType.Title,SubmissionStatus.IsLive,SubmissionStatus.ID,SubmissionStatus.Title,AddressInfo.ID,AddressInfo.Title,AddressInfo.AddressLine1,AddressInfo.AddressLine2,AddressInfo.Town,AddressInfo.StateOrProvince,AddressInfo.Postcode,AddressInfo.CountryID,AddressInfo.Country.ISOCode,AddressInfo.Country.ContinentCode,AddressInfo.Country.ID,AddressInfo.Country.Title,AddressInfo.Latitude,AddressInfo.Longitude,AddressInfo.ContactTelephone1,AddressInfo.ContactTelephone2,AddressInfo.ContactEmail,AddressInfo.AccessComments,AddressInfo.RelatedURL,AddressInfo.Distance,AddressInfo.DistanceUnit,OperatorInfo,UsageType
0,None,None,None,True,2021-10-13T09:40:00Z,189571,026932A1-E14A-4892-820E-1668C9AD5BE9,None,1,None,108.0,None,1.0,"€0,37/kWh","[{'ID': 306650, 'ConnectionTypeID': 25, 'ConnectionType': {'FormalName': 'IEC 62196-2 Type 2', 'IsDiscontinued': False, 'IsObsolete': False, 'ID': 25, 'Title': 'Type 2 (Socket Only)'}, 'Reference': '6262953e-3e32-43d9-abf8-5a8f6dc792e5', 'StatusTypeID': 50, 'StatusType': {'IsOperational': True, 'IsUserSelectable': True, 'ID': 50, 'Title': 'Operational'}, 'LevelID': 2, 'Level': {'Comments': 'Over 2 kW, usually non-domestic socket type', 'IsFastChargeCapable': False, 'ID': 2, 'Title': 'Level 2 : Medium (Over 2kW)'}, 'Amps': 16, 'Voltage': 400, 'PowerKW': 11.0, 'CurrentTypeID': 20, 'CurrentType': {'Description': 'Alternating Current - Three Phase', 'ID': 20, 'Title': 'AC (Three-Phase)'}, 'Quantity': 2, 'Comments': None}]",1.0,None,None,None,50,2021-10-13T11:34:00Z,None,1.0,2021-10-13T09:40:00Z,200,http://openchargemap.org,None,True,1,Manual Data Entry,False,True,True,Licensed under Creative Commons Attribution-ShareAlike 4.0 International (CC BY-SA 4.0),None,1,Open Charge Map Contributors,http://www.vattenfall.de/de/emobility/emobility.htm,None,0800 - 2 335 335,None,False,NaN,NaN,emobility@vattenfall.de,None,False,108.0,Vattenfall InCharge,None,None,None,1.0,Public,True,True,50,Operational,True,200,Submission Published,189930,Amsteldijk 707,Amsteldijk 707,Oost,Amsterdam,North Holland,1074JH,159,NL,EU,159,Netherlands,52.356967,4.904728,None,None,None,None,None,None,0,NaN,NaN
1,None,None,None,True,2021-10-12T22:57:00Z,189568,854DCF4B-2DB4-4356-BB0A-D4DABABB025F,None,1,None,103.0,None,4.0,"€0,37/kWh","[{'ID': 306643, 'ConnectionTypeID': 25, 'ConnectionType': {'FormalName': 'IEC 62196-2 Type 2', 'IsDiscontinued': False, 'IsObsolete': False, 'ID': 25, 'Title': 'Type 2 (Socket Only)'}, 'Reference': 'NLALLEGO001348', 'StatusTypeID': 50, 'StatusType': {'IsOperational': True, 'IsUserSelectable': True, 'ID': 50, 'Title': 'Operational'}, 'LevelID': 2, 'Level': {'Comments': 'Over 2 kW, usually non-domestic socket type', 'IsFastChargeCapable': False, 'ID': 2, 'Title': 'Level 2 : Medium (Over 2kW)'}, 'Amps': 

### Correct geschreven gemeente en provincie data ophalen

In [4]:
# https://download.geoportaaloverijssel.nl/download/vector/6be88637-f10f-44a4-aa5c-fc8c0f857620
df_gp = pd.read_csv('Grenzen_van_alle_Nederlandse_gemeenten_en_provincies.csv')
df_gp = df_gp[['SHAPE', 'POLY_AREA', 'PROVINCIENAAM', 'GEMEENTENAAM']]
df_gp.head()

SHAPE     POLY_AREA  PROVINCIENAAM  GEMEENTENAAM
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [ ]:
df_gp['geometry'] = gpd.GeoSeries.from_wkt(df_gp['SHAPE'])
gdf_gp = gpd.GeoDataFrame(df_gp, geometry='geometry')
gdf_gp.plot()

### Cleaning
#### Replace empty values with None

In [6]:
df_ocm = df_ocm_raw.copy()
# df_ocm = df_ocm.replace(np.nan, None)
df_ocm = df_ocm.replace('', np.nan)

#### Connections kolom is verbeterd
Nadat de request json was opgezet, hebben we gemerkt dat de kolom 'Connections' niet goed is genormaliseerd
naar een dataframe en nog JSON data bevat. Daarom is er gekozen om een **nested dataframe** te maken waarbij
de kolom 'Connections' een dataframe op zichzelf is.

In [7]:
df_ocm['Connections'] = df_ocm.apply(lambda row: pd.json_normalize(row['Connections']), axis=1)
display(df_ocm['Connections'].head())

0                                                                                                                                                       ID  ConnectionTypeID                             Reference  StatusTypeID  LevelID  Amps  Voltage  PowerKW  CurrentTypeID  Quantity Comments ConnectionType.FormalName  ConnectionType.IsDiscontinued  ConnectionType.IsObsolete  ConnectionType.ID  ConnectionType.Title  StatusType.IsOperational  StatusType.IsUserSelectable  StatusType.ID StatusType.Title                               Level.Comments  Level.IsFastChargeCapable  Level.ID                  Level.Title            CurrentType.Description  CurrentType.ID CurrentType.Title
0  306650                25  6262953e-3e32-43d9-abf8-5a8f6dc792e5            50        2    16      400     11.0             20         2     None        IEC 62196-2 Type 2                          False                      False                 25  Type 2 (Socket Only)                      True                   

#### Gemeente en provincie kolom opschonen
Tijdens het analyseren van de data, zijn we achter gekomen dat we veel data missen in de dataset.
De dataset heeft **7783** regels aan data.

Van de **7783** regels de Provincie kolom, zijn maar **296** waarden ingevuld en de overige **7487** waarden ontbreekt.

Van de **7783** regels de Gemeente kolom, zijn maar **7766** waarden ingevuld en de overige **17** waarden ontbreekt.

Nadat we de Gemeente waarden hebben gecorrigeerd, zien we dat **504** waarden verkeerd zijn ingevuld.
De hoofdreden voor deze verkeerde waarden is dat ze stedennamen of dorpsnamen hebben ingevuld die niet
gelijkstaan aan de gemeentenaam. Ook zijn de Provincie kolom aangevuld met de correcte Gemeente waarden.

Tot slot hebben we de Provincie kolom extra gecontroleerd op mogelijke vermiste waarden en hiermee hebben
we maar **444** vermiste waarden in Provincie.

In [8]:
# display(df_ocm['AddressInfo.Town'].value_counts(dropna=False))
print(f"De Data Tabel heeft {df_ocm.shape[0]} regels.")
print(f"De Provincie kolom heeft {df_ocm['AddressInfo.StateOrProvince'].notna().sum()} ingevulde waardes.")
print(f"De Provincie kolom heeft {df_ocm['AddressInfo.StateOrProvince'].isna().sum()} ontbrekende waardes.")
print(f"De Gemeente kolom heeft {df_ocm['AddressInfo.Town'].notna().sum()} ingevulde waardes.")
print(f"De Gemeente kolom heeft {df_ocm['AddressInfo.Town'].isna().sum()} ontbrekende waardes.")

De Data Tabel heeft 7785 regels.
De Provincie kolom heeft 298 ingevulde waardes.
De Provincie kolom heeft 7487 ontbrekende waardes.
De Gemeente kolom heeft 7768 ingevulde waardes.
De Gemeente kolom heeft 17 ontbrekende waardes.


In [9]:
df_ocm['cleaned.town'] = np.nan
df_ocm['cleaned.province'] = np.nan

In [ ]:
def get_cleaned_province_and_town(row):
    if row['AddressInfo.Town'] is None: return row
    if row['AddressInfo.Town'] is np.nan: return row

    highest_score_town = 0
    highest_score_gp = None
    for i,gp in df_gp.iterrows():
        ratio_town = fuzz.WRatio(gp['GEMEENTENAAM'],row['AddressInfo.Town'])
        if ratio_town > highest_score_town:
            highest_score_town = ratio_town
            highest_score_gp = gp

    if highest_score_town >= 90:
        row['cleaned.town'] = highest_score_gp['GEMEENTENAAM']
        row['cleaned.province'] = highest_score_gp['PROVINCIENAAM']
    return row

def get_extra_cleaned_province(row):
    if type(row['AddressInfo.StateOrProvince']) == str and type(row['cleaned.province']) == float:
        highest_score_province = 0
        highest_score_province_name = None
        for pro in df_gp['PROVINCIENAAM'].unique():
            ratio_province = fuzz.WRatio(pro,row['AddressInfo.StateOrProvince'])
            if ratio_province > highest_score_province:
                highest_score_province = ratio_province
                highest_score_province_name = pro
        if highest_score_province >= 70:
            row['cleaned.province'] = highest_score_province_name
    return row

df_ocm = df_ocm.apply(lambda row: get_cleaned_province_and_town(row), axis=1)
df_ocm = df_ocm.apply(lambda row: get_extra_cleaned_province(row), axis=1)

display(df_ocm['cleaned.town'].isna().sum())
display(df_ocm['cleaned.province'].isna().sum())
# display(df_ocm['cleaned.town'].value_counts(dropna=False))
# display(df_ocm['cleaned.province'].value_counts(dropna=False))
display(df_ocm[['AddressInfo.Town', 'cleaned.town', 'AddressInfo.StateOrProvince', 'cleaned.province']])

### Open Charge Map kolommen

In [ ]:
df_ocm.columns

### Aantal laadpunten per provincie

In [ ]:
df_ocm.groupby('AddressInfo.StateOrProvince')['NumberOfPoints'].sum()

#### Schoongemaakte aantal laadpunten per provincie

In [ ]:
print(df_ocm[df_ocm['AddressInfo.Town'].isna()])

In [ ]:
print(df_ocm['AddressInfo.StateOrProvince'].value_counts(dropna=False))
print(df_ocm['cleaned.province'].value_counts(dropna=False))

### Aantal laadpunten per gemeente

In [ ]:
df_ocm.groupby('AddressInfo.Town')['NumberOfPoints'].sum()

### Type laadpalen

In [ ]:
types = []
for con in df_ocm['Connections']:
    if 'CurrentType.Title' in con.columns:
        for title in con['CurrentType.Title']:
            types.append(title)
    else:
        types.append(np.nan)

unique, counts = np.unique(types, return_counts=True)

df_types = pd.DataFrame()
df_types['type'] = unique
df_types['count'] = counts
df_types.head()

### Zijn tarieven transparant?

- De meeste laadpalen zijn vaag met de specifieke tarieven
- De specificeerde tarieven zijn inconsistent met de andere laadpalen

In [ ]:
df_ocm['UsageCost'].value_counts()

### Kaart van Nederland met alle laadpunten

In [ ]:
fig = go.Figure(data=go.Scattergeo(
        lon = df_ocm['AddressInfo.Longitude'],
        lat = df_ocm['AddressInfo.Latitude'],
        text = df_ocm['AddressInfo.Title'],
        mode = 'markers',
        ))
fig.update_layout(
        title = 'Laadpalen in Nederland',
        geo_scope='europe',
    )
fig.show()